In [21]:
import pandas as pd

In [22]:
# 物性データをcsvから読み込む
# 2行分は不要なので3行目をヘッダーとして読み込む
df_properties = pd.read_csv('properties.csv', header=2)
# 時系列データをcsvから読み込む
df_sample_data = pd.read_csv('water_air_data_rows63936.csv')

# 水の比熱 [J/kg・K]
cp_water = df_properties['WaterCp'][0]
# 水の密度 [kg/m3]
rho_water=  df_properties['WaterRho'][0]
# 大気圧 [hPa]
air_p = df_properties['AtmosphericPressure'][0]

# 経過時間 [s]
time = df_sample_data['Time']
# 冷却水流量 [L/min]
flow_rate = df_sample_data['FlowRate']
# 冷却水入り温度 [℃]
inlet_temp = df_sample_data['InletTemperature']
# 冷却水戻り温度 [℃]
outlet_temp = df_sample_data['OutletTemperature']
# 気温 [℃]
air_temp = df_sample_data['AirTemperature']
# 相対湿度 [%]
relative_humidity = df_sample_data['RelativeHumidity']

In [30]:
%%time
# 行数取得
num_rows = len(df_sample_data)
# 列数取得
num_columns = len(df_sample_data.columns)

# 行数と同じ要素数の配列を用意
# 冷却水による脱熱量
heat_loss = [float(1.0) for i in range(num_rows)]
# 容積絶対湿度 [g/m3]
volumetric_humidity = [float(1.0) for i in range(num_rows)]
# 重量絶対湿度 [kg/kg(DA)]
mixing_ratio = [float(1.0) for i in range(num_rows)]

for i in range(num_rows):
    # 冷却水の出入りの温度差
    del_temp = outlet_temp[i] - inlet_temp[i]
    # 冷却水の脱熱量
    heat_loss[i] = flow_rate[i] / 1000 / 60 * cp_water * rho_water * del_temp # [W]
    # 飽和水蒸気圧 [hPa]
    es = 6.11 * 10 ** (7.5 * air_temp[i] / (237.3 + air_temp[i]))
    # 水蒸気分圧 [hPa]
    e = es * relative_humidity[i] / 100
    volumetric_humidity[i] = 217 * e / (273.15 + air_temp[i])
    mixing_ratio[i] = 0.62198 * e / (air_p - e)

Wall time: 1.52 s


In [31]:
# DataFrameに計算した変数の配列を結合
df_sample_data['HeatLoss'] = heat_loss
df_sample_data['VolumetricHumidity'] = volumetric_humidity
df_sample_data['MixingRatio'] = mixing_ratio
# CSVに書き込み
df_sample_data.to_csv(f'sample_output_rows{num_rows}_python.csv')